# Hybrid CNN + Quantum Neural Network for CIFAR-10
Implements a density QNN architecture based on the paper's framework

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pennylane as qml
import numpy as np

## Environment Setup
Load AWS configuration from .env file

In [12]:
import os
from dotenv import load_dotenv
from IPython.display import clear_output

# Load environment variables from .env file
load_dotenv()

# Function to safely get and mask sensitive environment variables
def get_masked_env(var_name):
    value = os.getenv(var_name, '')
    if value and var_name in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY']:
        return value[:4] + '*' * (len(value) - 8) + value[-4:]
    return value

# Display current configuration
print("Current AWS Configuration:")
print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")

# Function to update AWS configuration
def update_aws_config(region=None, access_key=None, secret_key=None, device_arn=None):
    if region:
        os.environ['AWS_DEFAULT_REGION'] = region
    if access_key:
        os.environ['AWS_ACCESS_KEY_ID'] = access_key
    if secret_key:
        os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
    if device_arn:
        os.environ['BRAKET_DEVICE'] = device_arn
    
    clear_output()
    print("Updated AWS Configuration:")
    print(f"Region: {get_masked_env('AWS_DEFAULT_REGION')}")
    print(f"Access Key ID: {get_masked_env('AWS_ACCESS_KEY_ID')}")
    print(f"Secret Access Key: {get_masked_env('AWS_SECRET_ACCESS_KEY')}")
    print(f"Braket Device: {get_masked_env('BRAKET_DEVICE')}")


Current AWS Configuration:
Region: us-east-1
Access Key ID: AKIA************ZMTY
Secret Access Key: RXLx********************************i0yI
Braket Device: arn:aws:braket:::device/quantum-simulator/amazon/sv1


In [13]:
# Load CIFAR-10 sample
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
images, labels = next(iter(trainloader))

print(f"Loaded batch: {images.shape}")

Files already downloaded and verified
Loaded batch: torch.Size([4, 3, 32, 32])
Loaded batch: torch.Size([4, 3, 32, 32])


In [16]:
# Amazon Braket SV1 simulator device
dev = qml.device(
    "braket.aws.qubit",
    device_arn=os.getenv('BRAKET_DEVICE'),
    wires=4,
    shots=100  # Set number of shots here
)

In [17]:
# Density QNN: Sub-unitary quantum circuit (RBS-based)
@qml.qnode(dev, interface="torch")
def quantum_sub_circuit(inputs, weights):
    # Data encoding
    for i in range(4):
        qml.RY(inputs[i], wires=i)
    
    # Parameterized RBS-inspired gates
    for i in range(4):
        qml.RZ(weights[i], wires=i)
    
    # Entanglement (CNOT ladder)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])
    
    # Second rotation layer
    for i in range(4):
        qml.RY(weights[i+4], wires=i)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

In [22]:
# Hybrid CNN + Density Quantum Model
class HybridDensityQNN(nn.Module):
    def __init__(self, num_sub_unitaries=2):
        super(HybridDensityQNN, self).__init__()
        
        # CNN feature extractor
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 4)
        
        # Density QNN: K sub-unitaries with independent parameters
        self.K = num_sub_unitaries
        self.quantum_layers = nn.ModuleList([
            qml.qnn.TorchLayer(quantum_sub_circuit, {"weights": (8,)})
            for _ in range(self.K)
        ])
        
        # Trainable mixing coefficients α_k (ensure sum to 1 via softmax)
        self.alpha = nn.Parameter(torch.ones(self.K))
        
        # Final classifier
        self.fc2 = nn.Linear(4, 10)
    
    def forward(self, x):
        # CNN feature extraction
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.tanh(self.fc1(x))
        
        # Density QNN: weighted sum of sub-unitaries
        alpha_norm = torch.softmax(self.alpha, dim=0)
        quantum_out = sum(alpha_norm[k] * self.quantum_layers[k](x) for k in range(self.K))
        
        # Classification
        out = self.fc2(quantum_out)
        return out

In [23]:
# Initialize and test forward pass
model = HybridDensityQNN(num_sub_unitaries=2)
output = model(images)

print(f"Input shape: {images.shape}")
print(f"Output shape: {output.shape}")
print(f"Mixing coefficients α: {torch.softmax(model.alpha, dim=0).detach()}")
print(f"\nOutput logits:\n{output}")

Input shape: torch.Size([4, 3, 32, 32])
Output shape: torch.Size([4, 10])
Mixing coefficients α: tensor([0.5000, 0.5000])

Output logits:
tensor([[ 0.2526, -0.3160,  0.0282,  0.6928,  0.0157, -0.7796, -0.5577,  0.0707,
          0.1227, -0.4648],
        [ 0.2553, -0.3378,  0.0234,  0.6888,  0.0476, -0.7318, -0.5319,  0.0629,
          0.0822, -0.4633],
        [ 0.2495, -0.3601,  0.0563,  0.6795, -0.0025, -0.7777, -0.5845,  0.0661,
          0.1195, -0.4718],
        [ 0.2814, -0.3382,  0.0272,  0.7245,  0.0389, -0.7296, -0.5493,  0.0232,
          0.0646, -0.4749]], grad_fn=<AddmmBackward0>)


## Output Analysis

Let's analyze the model's output and performance:

In [24]:
# 1. Analyze Predictions
probabilities = torch.softmax(output, dim=1)
predictions = torch.argmax(probabilities, dim=1)

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

print("Prediction Analysis:")
print("-" * 50)
for i in range(len(predictions)):
    true_class = classes[labels[i]]
    pred_class = classes[predictions[i]]
    confidence = probabilities[i][predictions[i]] * 100
    
    print(f"Image {i + 1}:")
    print(f"True class: {true_class}")
    print(f"Predicted: {pred_class} (Confidence: {confidence:.2f}%)")
    print(f"Top 3 predictions:")
    
    # Get top 3 predictions
    top3_prob, top3_idx = torch.topk(probabilities[i], 3)
    for j in range(3):
        print(f"  {classes[top3_idx[j]]}: {top3_prob[j]*100:.2f}%")
    print()

Prediction Analysis:
--------------------------------------------------
Image 1:
True class: plane
Predicted: cat (Confidence: 20.15%)
Top 3 predictions:
  cat: 20.15%
  plane: 12.97%
  ship: 11.39%

Image 2:
True class: horse
Predicted: cat (Confidence: 20.08%)
Top 3 predictions:
  cat: 20.08%
  plane: 13.02%
  ship: 10.95%

Image 3:
True class: dog
Predicted: cat (Confidence: 20.04%)
Top 3 predictions:
  cat: 20.04%
  plane: 13.04%
  ship: 11.45%

Image 4:
True class: horse
Predicted: cat (Confidence: 20.76%)
Top 3 predictions:
  cat: 20.76%
  plane: 13.33%
  ship: 10.73%



In [29]:
# 2. Analyze Quantum Layer Behavior
print("Quantum Layer Analysis:")
print("-" * 50)

# Analyze mixing coefficients
alpha_norm = torch.softmax(model.alpha, dim=0)
print("Sub-unitary mixing coefficients:")
for k in range(model.K):
    print(f"α_{k + 1}: {alpha_norm[k]:.4f}")
print()

# Get quantum outputs for first image
with torch.no_grad():
    # Extract features
    x = images[0:1]  # Add batch dimension
    x = model.pool(torch.relu(model.conv1(x)))
    x = model.pool(torch.relu(model.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    angles = torch.tanh(model.fc1(x)) * 3.1415
    
    # Get outputs from each quantum layer
    quantum_outputs = [layer(angles[0]) for layer in model.quantum_layers]
    
print("Quantum circuit outputs for first image:")
for k in range(model.K):
    print(f"Circuit {k + 1} output: {quantum_outputs[k]}")
print(f"\nWeighted sum: {sum(alpha_norm[k] * quantum_outputs[k] for k in range(model.K))}")

Quantum Layer Analysis:
--------------------------------------------------
Sub-unitary mixing coefficients:
α_1: 0.5000
α_2: 0.5000

Quantum circuit outputs for first image:
Circuit 1 output: tensor([ 0.6200, -1.0000, -1.0000,  0.4400])
Circuit 2 output: tensor([-0.6600, -0.8400, -0.6200, -0.6200])

Weighted sum: tensor([-0.0200, -0.9200, -0.8100, -0.0900], grad_fn=<AddBackward0>)
Quantum circuit outputs for first image:
Circuit 1 output: tensor([ 0.6200, -1.0000, -1.0000,  0.4400])
Circuit 2 output: tensor([-0.6600, -0.8400, -0.6200, -0.6200])

Weighted sum: tensor([-0.0200, -0.9200, -0.8100, -0.0900], grad_fn=<AddBackward0>)


In [ ]:
# 3. Visualize Feature Spaces
import matplotlib.pyplot as plt

with torch.no_grad():
    # Extract pre-quantum features
    x = model.pool(torch.relu(model.conv1(images)))
    x = model.pool(torch.relu(model.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    features = model.fc1(x)
    
    # Get quantum outputs
    x = torch.tanh(features)
    alpha_norm = torch.softmax(model.alpha, dim=0)
    
    quantum_outputs = []
    for i in range(len(x)):
        circuit_outputs = [layer(x[i:i+1]) for layer in model.quantum_layers]
        quantum_out = sum(alpha_norm[k] * circuit_outputs[k] for k in range(model.K))
        quantum_outputs.append(quantum_out.squeeze())
    quantum_outputs = torch.stack(quantum_outputs)

# Convert to numpy and get numeric labels
features_np = features.numpy()
quantum_np = quantum_outputs.numpy()
labels_np = labels.numpy()  # CIFAR labels are already numeric 0-9

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

scatter1 = ax1.scatter(features_np[:, 0], features_np[:, 1], c=labels_np, cmap='tab10', s=100)
ax1.set_title('Pre-Quantum Feature Space', fontsize=14)
ax1.set_xlabel('Feature Dim 1')
ax1.set_ylabel('Feature Dim 2')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='Class')

scatter2 = ax2.scatter(quantum_np[:, 0], quantum_np[:, 1], c=labels_np, cmap='tab10', s=100)
ax2.set_title('Quantum Feature Space', fontsize=14)
ax2.set_xlabel('Qubit 0 ⟨Z⟩')
ax2.set_ylabel('Qubit 1 ⟨Z⟩')
ax2.grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=ax2, label='Class')

plt.tight_layout()
plt.show()

# Statistics
print("\nFeature Statistics:")
print("-" * 50)
print(f"Pre-Quantum range: [{features.min():.3f}, {features.max():.3f}]")
print(f"Quantum range: [{quantum_outputs.min():.3f}, {quantum_outputs.max():.3f}]")
print(f"\nPre-Quantum mean±std: {features.mean():.3f}±{features.std():.3f}")
print(f"Quantum mean±std: {quantum_outputs.mean():.3f}±{quantum_outputs.std():.3f}")

AttributeError: 'list' object has no attribute 'numpy'